# Import packages and Data

In [1]:
import pandas as pd
#import matplotlib
#from tabulate import tabulate

Import visual condition data

In [2]:
df_vi = pd.read_excel(r'C:\Users\robert.everitt\OneDrive - National Grid\Data Engineering Course\Data\VI_E0309B_Test.xlsx')

#remove leading and trailing whitespace
df_vi.columns = df_vi.columns.str.strip()

Import importance scores for questions

In [3]:
df_importance = pd.read_excel(r'C:\Users\robert.everitt\OneDrive - National Grid\Data Engineering Course\Data\Importance_Scores.xlsx')

#remove leading and trailing whitespace
df_importance.columns = df_importance.columns.str.strip()

import expected script versions

In [4]:
df_exp_script_versions = pd.read_excel(r'C:\Users\robert.everitt\OneDrive - National Grid\Data Engineering Course\Data\Script_Versions.xlsx')

df_exp_script_versions.columns = df_exp_script_versions.columns.str.strip()

# Define Functions

Define a function to Find the first row containing 'search_text', remove rows above it, and set it as column names.

In [8]:
def clean_dataframe(df, search_text):

    # Find the index of the first occurrence of the search text
    idx = df[df.apply(lambda row: row.astype(str).str.contains(search_text, case=False).any(), axis=1)].index
    
    if idx.empty:
        raise ValueError(f"'{search_text}' not found in the DataFrame.")
    
    # Get the first matching index
    first_match_idx = idx[0]
    
    # Update the DataFrame by removing rows above the match
    df = df.iloc[first_match_idx:].reset_index(drop=True)
    
    # Set the first row as column names and drop it from the DataFrame
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)
    
    return df

Define a function to check if Script Activity IDs all have unique date

In [9]:
def check_consistent_dates(df, reference_col, date_col):
    """
    Checks if a given reference number always corresponds to the same date
    in a Pandas DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        reference_col (str): The name of the column containing reference numbers.
        date_col (str): The name of the column containing dates.

    Returns:
        dict: A dictionary indicating consistency.
              - If all consistent: {'consistent': True}
              - If inconsistent: {'consistent': False, 'violating_references': list_of_references}
    """
    # Group by the reference column and count the number of unique dates for each reference
    date_counts = df.groupby(reference_col)[date_col].nunique()

    # Identify reference numbers that have more than one unique date
    violating_references = date_counts[date_counts > 1].index.tolist()

    if not violating_references:
        return {'consistent': True}
    else:
        return {'consistent': False, 'violating_references': violating_references}

Define a function to return the maximum script version or each script in the visual condition data, for comparision with those in df_Script_Versions. If script version in data is higher, this will need checking to ensure there is still an importance score for all questions

In [10]:
def get_max_script_versions(df):
    # Get unique Script IDs
    unique_script_ids = df['Script ID'].unique()
    
    # Initialize a dictionary to store results
    max_versions = {}
    
    # Loop through each unique Script ID
    for script_id in unique_script_ids:
        # Filter the DataFrame for the current Script ID
        filtered_df = df[df['Script ID'] == script_id]
        
        # Find the maximum Script Version Number
        max_version = filtered_df['Script Version Number'].max()
        
        # Store the result in the dictionary
        max_versions[script_id] = max_version
    
    # Convert the dictionary to a DataFrame
    result_df = pd.DataFrame(list(max_versions.items()), columns=['Script ID', 'Script Version Number'])
    
    return result_df


# Main Code

User defined function to remove blank rows at the top and set columns headers

In [11]:
df_vi_clean = clean_dataframe(df_vi,'Location Code')

Define column types

In [12]:
date_columns = ['Inspection Date']
int_columns = ['Response Score', 'Script Item Number']
string_columns = ['Location Code','Plant Number','Installed Component','Installed Modifier','Script ID', 
                  'Script Description','Attribute ID','Questions','Response Description',
                  'Script Activity ID','Script Result ID','Site','Work Group','Zone','Script Version Number']

Convert data types based on column types above

In [13]:
for col in date_columns:
    df_vi_clean[col] = df_vi_clean[col].astype(str)
    df_vi_clean[col] = pd.to_datetime(df_vi_clean[col], errors='coerce').dt.strftime('%d/%m/%Y')

for col in int_columns:
    df_vi_clean[col] = pd.to_numeric(df_vi_clean[col], errors='coerce')

for col in string_columns:
    df_vi_clean[col] = df_vi_clean[col].astype(str, errors='ignore')

Create a reduced version of the cleaned visual inspection dataframe, by combining only the columns needed into a new dataframe

In [15]:
# Combine all columns into one list
selected_columns = date_columns + int_columns + string_columns

# Select only the specified columns
df_vi_reduced = df_vi_clean[selected_columns]

Create dataframe of maximum script versions from the visual inspection data using user defined function

In [16]:
df_vi_script_versions = get_max_script_versions(df_vi_reduced)

Convert data in `Script Version Number` to a numeric value

In [17]:
df_vi_script_versions['Script Version Number'] = pd.to_numeric(df_vi_script_versions['Script Version Number'], errors='coerce')

Merge `df_vi_script_versions` and `df_Script_Versions` on `Script ID`. Look for any descrepancies where the maximum script version from the visual inspection data is greater than the expected maximum script version. 
Return either a list of descripancies, or confirmation that there are no descrepancies.
If there are descripancies, it need to be checked that there aren't any new questions on the updated scripts that don't have importance scores assigned

In [18]:
# Merge dataframes on 'Script ID'
merged_df = pd.merge(df_vi_script_versions, df_exp_script_versions, on='Script ID', suffixes=('_VI', '_Exp'))

# Check for version discrepancies
discrepancies = merged_df[merged_df['Script Version Number_VI'] > merged_df['Script Version Number_Exp']]

# Get the list of Script IDs where the condition is true
script_ids_with_issues = discrepancies['Script ID'].tolist()

# Output the result

if not script_ids_with_issues:
    print("The list of script IDs with issues is empty.")
else:
    print("The following script IDs have version issues.", script_ids_with_issues)
    
print("\n" + "="*50 + "\n")

The list of script IDs with issues is empty.




In [19]:
print(merged_df)

     Script ID  Script Version Number_VI  Script Version Number_Exp
0      RVIVTAA                         1                          6
1     RVISPIAA                         5                          6
2     RVIDISAA                         1                          8
3      RVIESAA                         1                          7
4     RVICSEAA                         1                         10
5      RVICTAA                         1                          8
6    RVIBUSHAA                         1                          1
7     RVIGCBAA                         1                          7
8      RVISAAA                         1                          5
9      RVIBHAA                         1                          7
10    RVIVCTAA                         1                          4
11   RVIABCBAA                         1                          8
12     RVINEAA                         1                          4
13  RVSBUILDAA                         2        

Create a copy of `df_vi_reduced` to avoid *SettingWithCopyWarning* error.
Then run user defined fucntion to check if each `Script Activity ID` has a unique date.
An insepction should take place on a single day, so `Script Activity ID`s with muptiple dates would suggest some duplication, which means `Script Activity ID` can't be used as a unique identifier

In [20]:
# Create a copy of df_reduced to avoid SettingWithCopyWarning
df_vi_reduced_copy = df_vi_reduced.copy()

# Call the function with the copied DataFrame
result_consistent = check_consistent_dates(df_vi_reduced_copy, 'Script Activity ID', 'Inspection Date')


In [21]:
if result_consistent['consistent']:
    print("Result: All reference numbers consistently have the same date.")
else:
    print(f"Result: The following reference numbers have inconsistent dates: {result_consistent['violating_references']}")
    print("\nRows with inconsistent dates:")
    print(df[df['reference_number'].isin(result_consistent['violating_references'])].sort_values(by='reference_number'))

print("\n" + "="*50 + "\n")

Result: All reference numbers consistently have the same date.




In [22]:
# Adding a new column with the first 5 characters of 'original_column'
df_vi_reduced_copy.loc[:,'5Char_Site_Code'] = df_vi_reduced_copy['Location Code'].str[:5]

df_vi_reduced_copy.loc[:,'Civil Item'] = df_vi_reduced_copy['Attribute ID'].str.contains('CIV').replace({True: 'Civil Item', False: None})



In [23]:
df_vi_reduced_copy.head()

,Inspection Date,Response Score,Script Item Number,Location Code,Plant Number,Installed Component,Installed Modifier,Script ID,Script Description,Attribute ID,Questions,Response Description,Script Activity ID,Script Result ID,Site,Work Group,Zone,Script Version Number,5Char_Site_Code,Civil Item
0,01/08/2022,1.0,1,SUND4,SUND4X30VT,VT,R,RVIVTAA,VT S/S VISUAL INSPECTION,VIGEN01,Is the asset available?,YES,S-0659829,SRR-10328009,SUND,TCNWANG,TCN,1,SUND4,None
1,01/08/2022,NaN,2,SUND4,SUND4X30VT,VT,R,RVIVTAA,VT S/S VISUAL INSPECTION,VIGEN02,Visually inspect general condition of the asse...,nan,S-0659829,SRR-10328010,SUND,TCNWANG,TCN,1,SUND4,None
2,01/08/2022,0.0,4,SUND4,SUND4X30VT,VT,R,RVIVTAA,VT S/S VISUAL INSPECTION,VIGEN03,Do you want to report any condition deteriorat...,NO,S-0659829,SRR-10328011,SUND,TCNWANG,TCN,1,SUND4,None
3,01/08/2022,NaN,5,SUND4,SUND4X30VT,VT,R,RVIVTAA,VT S/S VISUAL INSPECTION,VIGEN04,Please take a wide photograph of the asset,nan,S-0659829,SRR-10328012,SUND,TCNWANG,TCN,1,SUND4,None
4,01/08/2022,1.0,1,SUND4,SUND4X30VT,VT,Y,RVIVTAA,VT S/S VISUAL INSPECTION,VIGEN01,Is the asset available?,YES,S-0659838,SRR-10328029,SUND,TCNWANG,TCN,1,SUND4,None


In [24]:
# Specify the columns to check
check_column = 'Civil Item' 
target_column = 'Attribute ID' 

In [25]:
# Initialize a list to store the indices of matches
matches = []

In [26]:
# Iterate through the DataFrame
for i in range(len(df_vi_reduced_copy) - 1):  # -1 to avoid index out of range
    if df_vi_reduced_copy[check_column].iloc[i] == 'Civil Item':
        # Start checking the next rows for 'VIGEN05'
        j = i + 1
        while j < len(df_vi_reduced_copy) and df_vi_reduced_copy[target_column].iloc[j] == 'VIGEN05':
            matches.append(j)  # Store the index of 'VIGEN05'
            j += 1  # Move to the next row

for row in matches:
    df_vi_reduced_copy.at[row, check_column] = 'Civil Photo'

df_vi_civil = df_vi_reduced_copy[df_vi_reduced_copy['Civil Item'].notna()]

create another dataframe which contains only civil items

In [27]:
df_vi_civil_items = df_vi_civil[df_vi_civil['Civil Item'] == 'Civil Item']

df_vi_civil_items.head()

,Inspection Date,Response Score,Script Item Number,Location Code,Plant Number,Installed Component,Installed Modifier,Script ID,Script Description,Attribute ID,Questions,Response Description,Script Activity ID,Script Result ID,Site,Work Group,Zone,Script Version Number,5Char_Site_Code,Civil Item
11,01/08/2022,0.0,7,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVCN01,Rate condition of the foundation / base,"NO SIGNIFICANT DEFECT, AS GOOD AS NEW",S-0659897,SRR-10328410,SUND,TCNWANG,TCN,1,SUND4,Civil Item
13,01/08/2022,20.0,13,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVMT01,Rate corrosion on the steel asset support stru...,G2-COATING INTACT (<1%)/VERY LIGHT CORROSION,S-0659897,SRR-10328412,SUND,TCNWANG,TCN,1,SUND4,Civil Item
19,01/08/2022,0.0,26,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVGA09,Do any of the earthing connections need reporting,NO,S-0659897,SRR-10328416,SUND,TCNWANG,TCN,1,SUND4,Civil Item
20,01/08/2022,0.0,31,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVGA12,Does the asset body need reporting,NO,S-0659897,SRR-10328417,SUND,TCNWANG,TCN,1,SUND4,Civil Item
21,01/08/2022,0.0,45,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVGA12,Does the asset body need reporting,NO,S-0659897,SRR-10328418,SUND,TCNWANG,TCN,1,SUND4,Civil Item


In [31]:
# Define the valid response scores
valid_scores = [0, 20, 40, 60, 80, 100]

# Filter the DataFrame
df_scores = df_vi_civil_items[df_vi_civil_items['Response Score'].isin(valid_scores) & ~df_vi_civil_items['Response Description'].isin(['YES', 'NO', 'Non-Metallic']) & 
    df_vi_civil_items['Response Description'].notnull()]

df_scores.head()

,Inspection Date,Response Score,Script Item Number,Location Code,Plant Number,Installed Component,Installed Modifier,Script ID,Script Description,Attribute ID,Questions,Response Description,Script Activity ID,Script Result ID,Site,Work Group,Zone,Script Version Number,5Char_Site_Code,Civil Item
11,01/08/2022,0.0,7,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVCN01,Rate condition of the foundation / base,"NO SIGNIFICANT DEFECT, AS GOOD AS NEW",S-0659897,SRR-10328410,SUND,TCNWANG,TCN,1,SUND4,Civil Item
13,01/08/2022,20.0,13,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVMT01,Rate corrosion on the steel asset support stru...,G2-COATING INTACT (<1%)/VERY LIGHT CORROSION,S-0659897,SRR-10328412,SUND,TCNWANG,TCN,1,SUND4,Civil Item
31,01/08/2022,0.0,7,SUND4,SUND4X303,DIS,nan,RVIDISAA,DISC S/S VISUAL INSPECTION,VICIVCN01,Rate condition of the foundation / base,"NO SIGNIFICANT DEFECT, AS GOOD AS NEW",S-0659910,SRR-10328696,SUND,TCNWANG,TCN,1,SUND4,Civil Item
33,01/08/2022,0.0,11,SUND4,SUND4X303,DIS,nan,RVIDISAA,DISC S/S VISUAL INSPECTION,VICIVCN02,Rate condition of the concrete asset support s...,"NO SIGNIFICANT DEFECT, AS GOOD AS NEW",S-0659910,SRR-10328698,SUND,TCNWANG,TCN,1,SUND4,Civil Item
81,02/08/2022,0.0,7,SUND4,SUND4X124,DIS,nan,RVIDISAA,DISC S/S VISUAL INSPECTION,VICIVCN01,Rate condition of the foundation / base,"NO SIGNIFICANT DEFECT, AS GOOD AS NEW",S-0661012,SRR-10335318,SUND,TCNWANG,TCN,1,SUND4,Civil Item


**Explanation:**\
`isin` is used to exclude all the rows where `Response Score` is not one of the values in `valid_scores`  
Negation with ~: The ~ operator is used to negate the condition, meaning we want to exclude rows where 'Response Description' is in the specified list.\
`notnull()` Method: The `notnull()` method is used to ensure that we only keep rows where `Response Description` is not null.\
Combining Conditions: The conditions are combined using the & operator to ensure both criteria are met.\
The resulting DataFrame `df_scores` will contain only the rows that meet both filtering criteria.

Add a new column in df_final which concatenates script and question. This will be used to link the importance scores

In [32]:
# Concatenating two columns into a new column with a space in between

df_final = df_scores.copy()

df_final.loc[:,'Script and Question'] = df_final['Script Description'] + ' ' + df_final['Questions']


Merge df_final and df_importance to add in inportance scores associated with each script/queston

In [34]:
# Merging df_final with df_importance based on 'Script and Question'
df_final = df_final.merge(df_importance[['Script and Question', 'Importance score']], 
                          on='Script and Question', 
                          how='left')


Add in another column WCS_Element, which contains the componets for each question which will feed into the WCS equation for the whole asset.  
Add in another column which combines the componennt/question/scrit activity ID (**why?**)

In [ ]:
df_final['WCS_Element'] = 1-(df_final['Importance score']/100)*(df_final['Response Score']/100)
df_final['Component_Q_ScriptActID'] = df_final['Installed Component'] + df_final['Questions'] + df_final['Script Activity ID']

Create a new column to store the maximum Response Score for each Component_Q_ScriptActID, this will be used later to avoid the situation where we count muliple fence panels/trench covers etc. as concurrent defects on the same asset, leading to very high WCS scores

In [38]:
# Create a new column to store the maximum Response Score for each Component_Q_ScriptActID
df_final['Max_Response_Score'] = None

# Loop through each row in the DataFrame
for index, row in df_final.iterrows():
    # Get the current Component_Q_ScriptActID
    current_id = row['Component_Q_ScriptActID']
    
    # Find the maximum Response Score for the current Component_Q_ScriptActID
    max_score = df_final[df_final['Component_Q_ScriptActID'] == current_id]['Response Score'].max()
    
    # Assign the maximum score to the new column
    df_final.at[index, 'Max_Response_Score'] = max_score
        

...Add descriptions from here...

In [47]:

# Assuming df is your DataFrame
df_unique = df_final.drop_duplicates(subset='Component_Q_ScriptActID')


In [48]:
df_unique = df_unique.copy()

df_unique['WCS'] = 1- df_unique.groupby('Component_Q_ScriptActID')['WCS_Element'].transform('prod')

df_unique.head()

,Inspection Date,Response Score,Script Item Number,Location Code,Plant Number,Installed Component,Installed Modifier,Script ID,Script Description,Attribute ID,...,Zone,Script Version Number,5Char_Site_Code,Civil Item,Script and Question,Importance score,WCS_Element,Component_Q_ScriptActID,Max_Response_Score,WCS
0,01/08/2022,0.0,7,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVCN01,...,TCN,1,SUND4,Civil Item,VT S/S VISUAL INSPECTION Rate condition of the...,82.5,1.00,VT Rate condition of the foundation / baseS-06...,0.0,0.00
1,01/08/2022,20.0,13,SUND4,SUND4X30VT,VT,B,RVIVTAA,VT S/S VISUAL INSPECTION,VICIVMT01,...,TCN,1,SUND4,Civil Item,VT S/S VISUAL INSPECTION Rate corrosion on the...,75.0,0.85,VT Rate corrosion on the steel asset support s...,20.0,0.15
2,01/08/2022,0.0,7,SUND4,SUND4X303,DIS,nan,RVIDISAA,DISC S/S VISUAL INSPECTION,VICIVCN01,...,TCN,1,SUND4,Civil Item,DISC S/S VISUAL INSPECTION Rate condition of t...,85.0,1.00,DISRate condition of the foundation / baseS-06...,0.0,0.00
3,01/08/2022,0.0,11,SUND4,SUND4X303,DIS,nan,RVIDISAA,DISC S/S VISUAL INSPECTION,VICIVCN02,...,TCN,1,SUND4,Civil Item,DISC S/S VISUAL INSPECTION Rate condition of t...,75.0,1.00,DISRate condition of the concrete asset suppor...,0.0,0.00
4,02/08/2022,0.0,7,SUND4,SUND4X124,DIS,nan,RVIDISAA,DISC S/S VISUAL INSPECTION,VICIVCN01,...,TCN,1,SUND4,Civil Item,DISC S/S VISUAL INSPECTION Rate condition of t...,85.0,1.00,DISRate condition of the foundation / baseS-06...,0.0,0.00


In [49]:
# Merge df_final with df_unique on Component_Q_ScriptActID
df_final = df_final.merge(df_unique[['Script Activity ID', 'WCS']], 
                          on='Script Activity ID')


In [50]:
df_final.to_excel(r'C:\Users\robert.everitt\OneDrive - National Grid\Data Engineering Course\WCS_Test.xlsx')